In [1]:
# imports
import pandas as pd
import numpy as np
import random
import statistics as stat

traindf = pd.read_pickle("./pickles/df_small_train.pkl")

In [2]:
def random_solution(df):
    
    # csv_name = "./results/" + csv_name
    
    # get list of hotels per search id
    results_per_search = df.groupby('srch_id').size()
    
    column_values = df["srch_id"].values
    
    ids =  pd.unique(column_values) 
    
    all_ranks = []
    
    for i, ideetje in enumerate(ids):
        df_temp = df[df["srch_id"] == ideetje]
        
        ranks = df_temp["position"].values
        
        # shuffle ranks
        random.shuffle(ranks)
        all_ranks.extend(ranks)

    df["predicted_rank"] = all_ranks
    return df

In [3]:
def indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked):
    idcg = 0
    dcg = 0

    # itereer over elk resultaat in de search
    for i in range(len(true_rank)):
        # hotels die geboekt zijn zijn heel belangrijk (5)
        if booked[i] == 1:
            
            # increment ideal score
            if i != 0:
                idcg = idcg + (5 / np.log2(i+1))
            else:
                idcg += 5
            
            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                if i != 0:
                    dcg = dcg + (5 / np.log2(i+1))
                else:
                    dcg += 5
                
        # hotels waarop geklikt is zijn een beetje belangrijk (1)
        elif clicked[i] == 1:
            
            # increment ideal score
            if i != 0:
                idcg = idcg + (1 / np.log2(i+1))
            else:
                idcg += 1
            
            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                if i != 0:
                    dcg = dcg + (1 / np.log2(i+1))
                else:
                    dcg += 1
    
    return dcg/idcg

In [4]:
# score function for every search (thus iterate over groupby object)
def dataframe_ndcg_score(solution):
    all_scores = []
    for srchid, group in solution.groupby('srch_id'):

        # true_rank
        true_rank = np.arange(1,len(group)+1)

        # predicted_rank
        predicted_rank = np.asarray(group['predicted_rank'])

        # booking_bool
        booked = np.asarray(group['booking_bool'])

        # click_bool
        clicked = np.asarray(group['click_bool'])

        all_scores.append(indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked))
    return np.nanmean(all_scores)

In [5]:
print(dataframe_ndcg_score(random_solution(traindf)))

0.046578241539181
